In [2]:
# Import data
import pandas as pd
import numpy as np

# To show all data in dataframe
pd.set_option('display.max_colwidth', None)

raw = pd.read_json("data/kym.json")                   # 28799 meme templates from Know Your Meme, 16 features
events = pd.read_json("data/events.json")

In [4]:
memes = raw[raw['category'] == "Meme"] # Selecting only memes

In [1]:
# Look for Imgflip references                  
imgflip = {}
for title, d in zip(memes['title'], memes['additional_references']):
    for val in d.values():
        if "imgflip" in val:
            imgflip[title] = val
            
# print(imgflip)

NameError: name 'memes' is not defined

In [1]:
# Use pandasql to query data with SQL commands

!pip3 install -U pandasql
from pandasql import sqldf 

type_col = []
# Add separate column for type
for d in memes['details']:
    if 'type' in d.keys():
        t_list = [t.split("https://knowyourmeme.com/types/")[1] for t in d['type']]
        # type_col.append(t_list)
        type_col.append(", ".join(t_list))
    else:
        # type_col.append([])
        type_col.append("")
memes['type'] = type_col

# Select rows based on AND/OR query
mysql = lambda q: sqldf(q, globals())
test = memes[['type']]
mysql("SELECT * FROM test WHERE type LIKE '%snowclone%' AND type LIKE '%image-macro%' OR type LIKE '%cliche%';")

  Using cached pandasql-0.7.3-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.4 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
  Using cached pandas-1.4.1-cp39-cp39-macosx_11_0_arm64.whl (10.5 MB)
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.4.32-cp39-cp39-macosx_11_0_arm64.whl size=1535142 sha256=333653513af743b5d56ad1e1e7aff4c08b220c85cce271114a87fb4aae05c2f5
  Stored in directory: /Users/tarmopungas/Library/Caches/pip/wheels/f2/19/ac/766f9aa177238819a3fb79100dd2f7d9a348a6a082014afca2
Successfully built sqlalchemy


NameError: name 'memes' is not defined

In [1]:
# Siblings are weird

c = 0
for title, url, sib in zip(raw['title'], raw['url'], raw['siblings']):
    if type(sib) == list:
        print(title, url, sib)

    c += 1
    if c == 10:
        break

NameError: name 'raw' is not defined

In [2]:
# Web entity detection in Google's Vision AI data
vision.head()

c = 0
print(instances.keys())
print()
for instances in vision['webDetection']:
    if 'partialMatchingImages' in instances:
        print(instances['partialMatchingImages'])
    print()
    #print(instances['webEntities'])

    c += 1
    if c == 20:
        break

NameError: name 'vision' is not defined

# Visualizations

In [ ]:
# 